In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.mixture import BayesianGaussianMixture
from sklearn import model_selection

import random
import math
import datetime

In [2]:
dataset = 'ether'

In [3]:
if dataset == 'ibm':
    df_transactions = pd.read_csv("archive/User0_credit_card_transactions.csv")
    df_cards = pd.read_csv("archive/sd254_cards.csv")
    df_users = pd.read_csv("archive/sd254_users.csv")

    df_transactions_copy = df_transactions.copy() #IDK if we'll need this later

    df_transactions = df_transactions.drop(columns=["Is Fraud?"])
    transactions = df_transactions.to_numpy()
    cards = df_cards.to_numpy()
    users = df_users.to_numpy()

if dataset == 'ether':
    df_transactions = pd.read_csv('archive/transaction_dataset.csv')
    df_transactions_copy = df_transactions.copy()

    df_transactions = df_transactions.drop(columns=['Unnamed: 0', 'FLAG', 'Index'])

    transactions = df_transactions.to_numpy()

In [4]:
#we need to have a meeting about this
if dataset == "archive-ccibm":

    #need to convert the CSV text values into numbers

    #transaction data
    num_transaction_types = 0
    num_merchant_cities = 0
    num_merchant_states = 0
    num_errors = 0

    #user data
    num_genders = 0
    num_user_cities = 0
    num_user_states = 0

    #card data
    num_brands = 0
    num_card_types = 0

    #transaction data
    transaction_types = {}
    merchant_cities = {}
    merchant_states = {}
    errors = {}

    #user data
    genders = {}
    user_cities = {}
    user_states = {}

    #card data
    card_brands = {}
    card_types = {}

    zero_time = datetime.datetime.strptime("00:00", "%H:%M")

    X = np.ndarray((transactions.shape[0], transactions.shape[1] + df_cards.shape[1] + df_users.shape[1]), dtype=float)
    X_reference = np.ndarray(X.shape, dtype=object)

    for i in range(transactions.shape[0]):
        user = transactions[i, 0]
        card = transactions[i, 1]

        card_data = df_cards[(df_cards['User'] == user) & (df_cards['CARD INDEX'] == card)]

        #these values are already numbers; no need to change them
        #TODO: normalize some of them?
        X[i, 0:5] = transactions[i, 0:5]
        X[i, 8] = transactions[i, 8]
        X[i, 11:13] = transactions[i, 11:13]

        X_reference[i, :transactions.shape[1]] = transactions[i, :]

        #convert the time of date string into the number of minutes past midnight
        dtime = datetime.datetime.strptime(transactions[i, 5], "%H:%M")
        X[i, 5] = (dtime - zero_time).seconds//60

        #convert dollar amount from "$134.03" to 134.03
        X[i, 6] = float(transactions[i, 6][1:])

        #use ints to represent the different transaction types
        if not transactions[i, 7] in transaction_types:
            transaction_types[transactions[i, 7]] = num_transaction_types
            num_transaction_types += 1
        X[i, 7] = transaction_types[transactions[i, 7]]

        #use ints to represent the different cities
        if not transactions[i, 9] in merchant_cities:
            merchant_cities[transactions[i, 9]] = num_merchant_cities
            num_merchant_cities += 1
        X[i, 9] = merchant_cities[transactions[i, 9]]

        #use ints to represent the different states
        if not transactions[i, 10] in merchant_states:
            merchant_states[transactions[i, 10]] = num_merchant_states
            num_merchant_states += 1
        X[i, 10] = merchant_states[transactions[i, 10]]

        #use ints to represent errors
        if not transactions[i, 13] in errors:
            errors[transactions[i, 13]] = num_errors
            num_errors += 1
        X[i, 13] = 0 #errors[transactions[i, 13]]
        #I disabled this because it caused too many outliers

        X[i, 14:18] = users[user, 1:5] 
        X_reference[i, transactions.shape[1]:transactions.shape[1] + users.shape[1] - 1] = users[user, 1:]

        #use ints to represent gender 
        #(pretty sure the csv only has male/female, but don't want to cause any issues)
        if not users[user, 5] in genders:
            genders[users[user, 5]] = num_genders
            num_genders += 1
        X[i, 18] = genders[users[user, 5]]

        #evey address is unique, I'm just going to leave it blank
        X[i, 19] = 0

        #appartment; probably not relevant
        X[i, 20] = users[user, 7]

        #use ints to represent cities
        if not users[user, 8] in user_cities:
            user_cities[users[user, 8]] = num_user_cities
            num_user_cities += 1
        X[i, 21] = user_cities[users[user, 8]]

        #use ints to represent states
        if not users[user, 9] in user_states:
            user_states[users[user, 9]] = num_user_states
            num_user_states += 1
        X[i, 22] = user_states[users[user, 9]]

        #ZIP, lat, long
        X[i, 23:26] = users[user, 10:13]

        #per capita - ZIP
        X[i, 26] = float(users[user, 13][1:])

        #yearly income
        X[i, 27] = float(users[user, 14][1:])

        #debt
        X[i, 28] = float(users[user, 15][1:])

        #FICO, Num credit cards
        X[i, 29:31] = users[user, 16:19]

        card_data_np = card_data.to_numpy()
        X_reference[i, transactions.shape[1] + users.shape[1] - 1:transactions.shape[1] + users.shape[1] + card_data_np.shape[1] - 3] = card_data_np[0][2:]
        #turns out I could have been using dataframes, rather than np
        if i == 5011: print(i, user, card_data['Card Brand'])
        if not card_data['Card Brand'][card] in card_brands:
            card_brands[card_data['Card Brand'][card]] = num_brands
            num_brands += 1
        X[i, 31] = card_brands[card_data['Card Brand'][card]]

        #use ints to represent card types
        if not card_data['Card Type'][card] in card_types:
            card_types[card_data['Card Type'][card]] = num_card_types
            num_card_types += 1
        X[i, 32] = card_types[card_data['Card Type'][card]]

        X[i, 33] = card_data['Card Number'][card]

        #read expiry date as months
        X[i, 34] = float(card_data['Expires'][card][0:2]) + float(card_data['Expires'][card][3:8]) * 12
        
        X[i, 35] = card_data['CVV'][card]

        X[i, 36] = 0 if card_data['Has Chip'][card] == 'No' else 1

        X[i, 37] = card_data['Cards Issued'][card]

        X[i, 38] = float(card_data["Credit Limit"][card][1:])

        X[i, 39] = float(card_data['Acct Open Date'][card][0:2]) + float(card_data['Acct Open Date'][card][3:8]) * 12

        X[i, 40] = card_data['Year PIN last Changed'][card]

        X[i, 41] = 0 if card_data['Card on Dark Web'][card] == 'No' else 1

        #generate extra information section

        #time since last transaction (0 for first transatciont)
        if i > 0 and df_transactions['User'][i] == df_transactions['User'][i-1]:
            transaction_time = datetime.datetime(df_transactions["Year"][i], df_transactions["Month"][i], df_transactions["Day"][i], int(df_transactions["Time"][i][0:2]), int(df_transactions['Time'][i][3:]))
            last_transaction_time = datetime.datetime(df_transactions["Year"][i - 1], df_transactions["Month"][i - 1], df_transactions["Day"][i - 1], int(df_transactions["Time"][i - 1][0:2]), int(df_transactions['Time'][i - 1][3:]))
            relative_time = transaction_time - last_transaction_time
            # print(relative_time)
            X[i, 42] = relative_time.total_seconds()
        X_reference[i, 42] = float(X[i, 42])

        for j in range(X.shape[1]):
            if math.isnan(X[i, j]): X[i, j] = 0

        

        # print(transactions[i])    
        # print(X[i], X[i, 11], transactions[i, 11])

    for i in range(X.shape[0]):
        print(X[i])
        print(X_reference[i])
    # print(X[0])

In [5]:
if dataset == 'ether':
    unique_indices = {}
    unique_indices_values = {}

    X = np.ndarray(transactions.shape)

    for i in range(X.shape[0]):
        for j in range(X.shape[1]):
            if isinstance(transactions[i, j], int) or isinstance(transactions[i, j], float):
                X[i, j] = transactions[i, j]
            else:
                if not j in unique_indices:
                    unique_indices[j] = {}
                    unique_indices_values[j] = 0
                if not transactions[i, j] in unique_indices[j]:
                    unique_indices[j][transactions[i, j]] = unique_indices_values[j]
                    unique_indices_values[j] += 1
                X[i, j] = unique_indices[j][transactions[i, j]]
                # print(j, type(transactions[i, j]))
            
            if math.isnan(X[i, j]):
                X[i, j] = 0
    
    excluded_indices = [22]
    included_indices = [i for i in range(X.shape[1]) if i not in excluded_indices]
    print(included_indices)

    X = X[:, included_indices]


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47]


In [6]:
# singular_indices = [i for i in range(0, X.shape[1])]
# epsilon = 10

# for i in range(X.shape[0]):
#     for j in singular_indices:
#         if X[i, j] - X[0, j] > epsilon:
#             print(X[i, j], X[0, j])
#             singular_indices.remove(j)

# print(singular_indices, X[0, singular_indices], X[1, singular_indices])

# indices = [i for i in range(0, X.shape[1]) if i not in singular_indices]
# print(indices)

# X = X[:, indices]

In [7]:
# X_train, X_test = model_selection.train_test_split(X, test_size=0.4, shuffle=False)
train_size = 5000
X_train = X[:train_size]
X_test = X[train_size:]
print(X_train[0], X_train.shape[0])
print(X_test[0])

[0.00000000e+00 8.44260000e+02 1.09371000e+03 7.04785630e+05
 7.21000000e+02 8.90000000e+01 0.00000000e+00 4.00000000e+01
 1.18000000e+02 0.00000000e+00 4.58067850e+01 6.58951300e+00
 0.00000000e+00 3.12200000e+01 1.20068100e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 8.10000000e+02 8.65691093e+02
 5.86466675e+02 0.00000000e+00 2.65000000e+02 3.55885438e+07
 3.56031695e+07 0.00000000e+00 3.00000000e+01 5.40000000e+01
 0.00000000e+00 5.80000000e+01 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 1.50000000e+07
 2.65586148e+05 0.00000000e+00 1.68309984e+07 2.71779920e+05
 0.00000000e+00 0.00000000e+00 0.00000000e+00 3.90000000e+01
 5.70000000e+01 0.00000000e+00 0.00000000e+00] 5000
[4.97500000e+03 2.57000000e+00 7.40000000e-01 9.20000000e+00
 3.00000000e+00 2.00000000e+00 0.00000000e+00 2.00000000e+00
 3.00000000e+00 9.54699383e+02 1.04630062e+03 1.00050000e+03
 8.11889280e+01 1.08038479e+03 6.66999528e+02 0.00000000e+00
 0.00000000e+00 0.00000000e+00 5.

In [8]:
bgm = BayesianGaussianMixture(n_components=5, tol=0.001, max_iter=1000, verbose=True, random_state=0, n_init=1, weight_concentration_prior=10)
bgm.fit(X_train)
labels = bgm.predict(X_test)
probs = bgm.predict_proba(X_test)
log_probs = bgm.score_samples(X_test)

# for i in range(log_prob.shape[0]):
#     print(log_prob[i])

# print(labels)

# for i in range(labels.shape[0]):
#     print(i, labels[i])

Initialization 0
  Iteration 10
  Iteration 20
  Iteration 30
  Iteration 40
  Iteration 50
  Iteration 60
  Iteration 70
  Iteration 80
  Iteration 90
  Iteration 100
  Iteration 110
  Iteration 120
  Iteration 130
  Iteration 140
  Iteration 150
  Iteration 160
  Iteration 170
  Iteration 180
  Iteration 190
  Iteration 200
  Iteration 210
  Iteration 220
  Iteration 230
  Iteration 240
  Iteration 250
  Iteration 260
  Iteration 270
  Iteration 280
  Iteration 290
  Iteration 300
  Iteration 310
  Iteration 320
  Iteration 330
  Iteration 340
  Iteration 350
  Iteration 360
  Iteration 370
  Iteration 380
  Iteration 390
  Iteration 400
  Iteration 410
  Iteration 420
  Iteration 430
  Iteration 440
  Iteration 450
  Iteration 460
  Iteration 470
  Iteration 480
  Iteration 490
  Iteration 500
  Iteration 510
  Iteration 520
  Iteration 530
  Iteration 540
  Iteration 550
  Iteration 560
  Iteration 570
  Iteration 580
  Iteration 590
  Iteration 600
  Iteration 610
  Iteration 620


In [9]:
evil_labels = {}
good_labels = {}

def g(i):
    if not int(labels[i]) in good_labels:
        good_labels[int(labels[i])] = 0
    good_labels[int(labels[i])] += 1

def e(i):
    if not int(labels[i]) in evil_labels:
        evil_labels[int(labels[i])] = 0
    evil_labels[int(labels[i])] += 1


In [26]:
epsilon = -400

dynamic_epsilon = True
best = 0
best_value = 0

print(f'Max={max(log_probs)}, Min={min(log_probs)}')
if dynamic_epsilon:
    for test_epsilon in range(int(max(log_probs)), int(max(log_probs))-400, -5):
        # print(test_epsilon)
        frauds = 0
        fps = 1

        for i in range(X_test.shape[0]):
            flag = df_transactions_copy.iloc[i + train_size]["FLAG"]

            if log_probs[i] < test_epsilon:
                if flag == 1: frauds += 1
                else: fps += 1
        
        fscore = (frauds ** 1.3)/fps
        if fscore > best_value:
            epsilon = test_epsilon
            best_value = fscore

print(f'epsilon={epsilon}')
# # unlikelyhoods = [sum(probs[i]) - probs[i, labels[i]] for i in range(probs.shape[0])]

# for i in range(probs.shape[0]):
#     print(probs[i, labels[i]])

fp, tp, fn, tn = 0,0,0,0
pos = 0
neg = 0
flags = 0

for i in range(X_test.shape[0]):
    flag = df_transactions_copy.iloc[i + train_size]["FLAG"]

    if flag == 0:
        if log_probs[i] < epsilon: 
            fp += 1
            pos += 1
        else: 
            tn += 1
            neg += 1
    if flag == 1:
        flags += 1
        if log_probs[i] < epsilon: 
            tp += 1
            pos += 1
        else: 
            fn += 1
            neg += 1

print(f'FP:{fp}, TP:{tp}, FN:{fn}, TN:{tn}, pos:{pos}, neg:{neg}, flags:{flags}')
print(f'correctly classified fraud:{tp}/{flags} ({tp/flags*100}%), incorrectly classified normal: {fp}/{(X_test.shape[0]-flags)} ({fp/(X_test.shape[0]-flags)*100}%)')

# print(min(unlikelyhoods))

# print('+' * 100)

# for i in [ 4099,  4100,  4101,  4102,  4103,  4104,  4105,  4106,  4107,  4108,
#         4109,  4111,  4112,  9761,  9762, 13678, 13683, 13684, 13685, 13687,
#        13688, 13689, 13690, 13698, 13699, 17737, 17738]:
#     print(unlikelyhoods[i])
#     pass

# print(log_probs)

# for i in range(log_probs.shape[0]):
#     if log_probs[i] < epsilon:
#         print(i, log_probs[i])

# for i in [ 4099,  4100,  4101,  4102,  4103,  4104,  4105,  4106,  4107,  4108,
#         4109,  4111,  4112,  9761,  9762, 13678, 13683, 13684, 13685, 13687,
#        13688, 13689, 13690, 13698, 13699, 17737, 17738]:
#     # print(log_probs[i])
#     print(probs[i, labels[i]])

Max=-140.62492866761335, Min=-1984873433.8827677
epsilon=-210
FP:426, TP:1295, FN:884, TN:2236, pos:1721, neg:3120, flags:2179
correctly classified fraud:1295/2179 (59.430931620009176%), incorrectly classified normal: 426/2662 (16.003005259203608%)
